# Heart Rate Estimation using PPG (Convolutional Neural Networks Classification)

The data has to be downloaded from [here](https://archive.ics.uci.edu/ml/datasets/PPG-DaLiA), unzipped into the current folder. Different folders for each individual has to be in the same directory as this notebook.

In [2]:
import pickle
import numpy as np
from math import floor
from tqdm import tqdm
lengths=[]
activities = []
ppg = []
lines = []
X = []
for i in tqdm(range(1,16)):
   st ='S'+str(i)+'/'+'S'+str(i)+'.pkl'
   with open(st, 'rb') as f:
       temp = pickle.load(f, encoding='latin1')
   ppg = temp['signal']['wrist']['BVP'].tolist()
   qppg =[]
   lengths.append(int(len(ppg)/128)-3)
   ac = temp['activity']
   tempac =[]
   for l in range(int(floor(ac.shape[0]/8))-3):
       activities.append(max(ac[l*8:(l+4)*8]))
   #print(len(activities),len(temp['label']))
   for p in range(int(len(ppg)/128)-3):
       t = np.zeros(512)
       k = 0
       for j in range(p*128,(4+p)*128):
           t[k] = ppg[j][0]
           k = k+1
       qppg.append(t)
   X.extend(qppg)
   lines.extend(temp['label'])
   #activities.extend(temp['activity'].tolist())
   #print(i)


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [02:02<00:00,  8.16s/it]


In [3]:
import numpy as np
X=np.array(X)

In [4]:
Y = []
for i in lines:
    t = np.zeros(1)
    t[0]=i
    Y.append(t)
Y = np.array(Y)

In [5]:
from math import floor, ceil
y_class = []
sort_dict = {}
for rate in Y:
   fl = floor(rate/5)
   ce = ceil(rate/5)
   st = str(fl*5) + "-" + str(ce*5)
   y_class.append(st)
   sort_dict[st] = fl-8
 
y_class = np.array(y_class)

In [6]:
np.unique(y_class)

array(['100-105', '105-110', '110-115', '115-120', '120-125', '125-130',
       '130-135', '135-140', '140-145', '145-150', '150-155', '155-160',
       '160-165', '165-170', '170-175', '175-180', '180-185', '185-190',
       '40-45', '45-50', '50-55', '55-60', '60-65', '65-70', '70-75',
       '75-80', '80-85', '85-90', '90-95', '95-100'], dtype='<U7')

In [7]:
y_one_hot = []
for rate in y_class:
    temp = np.zeros(30)
    temp[sort_dict[rate]] = 1
    y_one_hot.append(temp)

In [8]:
y_one_hot = np.array(y_one_hot)

X and y_one_hot has the input and output data for training the model. The total data accounted for 64697 samples and each input sample is of size 512.

In [10]:
print(X.shape,y_one_hot.shape)

(64697, 512) (64697, 30)


The input and output data is spiltted into train and test set using sklearn's train-test split

In [9]:
import scipy
import numpy as np
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y_one_hot, test_size=0.10, random_state=42)

We've used keras to train the CNN model. The model has been defined with various layers as shown below and activated with relu for the hidden layers and compiled with Adam optimisation.

In [14]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv1D, Reshape, MaxPooling1D, Flatten

max_words=512
model = Sequential()
model.add(Dense(512,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(512,activation='relu'))
model.add(Reshape((1,4,128)))
model.add(Conv1D(256,3,activation='relu',input_shape=[4,128]))
model.add(Reshape((4,128)))
model.add(MaxPooling1D(pool_size=2))
model.add(Reshape((1,4,64)))
model.add(Conv1D(256,3,activation='relu',input_shape=[4,128]))
model.add(Reshape((4,128)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(512,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(30,activation='relu'))
model.add(Reshape(([30])))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(optimizer='Adam',loss='mse')
print(model.metrics_names)

[]


We also use callbacks with early stopping with a patience of 15 and Model checkpoint to save the model.

In [12]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

callbacks = [EarlyStopping(monitor='loss', patience=15),
             ModelCheckpoint('cnn_classifier_model.h5', save_best_only=True, 
                             save_weights_only=False)]

Run the below code to load the trained model and evaluate.

In [ ]:
batch_size = 64
epochs = 300

history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,callbacks=callbacks,validation_data=(X_test,y_test), verbose=1)
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

Run the below code to load the trained model and evaluate.

In [16]:
import keras
loaded=keras.models.load_model('cnn_classifier_model.h5')

Run the below code to get the accuracies for train and test sets.

In [17]:
score = loaded.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

102/102 [==============================] - 0s 4ms/step - loss: 0.2941 - accuracy: 0.9196


In [18]:
score = loaded.evaluate(X_train, y_train, batch_size=batch_size, verbose=1)

910/910 [==============================] - 4s 4ms/step - loss: 0.3299 - accuracy: 0.9083
